In [1]:
import knowledge_graph_pipeline as pipeline

# Build Knowledge Graph

In [2]:
sql_file = "schema_link_expert.sql"
parsed_schema = pipeline.parse_schema(sql_file)
parsed_schema

{'Location': {'columns': [{'name': 'zip_code',
    'type': 'VARCHAR(10)',
    'constraints': ['PRIMARY KEY']},
   {'name': 'country', 'type': 'VARCHAR(100)', 'constraints': ['NOT NULL']},
   {'name': 'createdAt',
    'type': 'DATETIME',
    'constraints': ['NOT NULL', 'DEFAULT NOW()']},
   {'name': 'updatedAt', 'type': 'DATETIME', 'constraints': []}],
  'primary_keys': [],
  'foreign_keys': []},
 'User': {'columns': [{'name': 'id',
    'type': 'BIGINT',
    'constraints': ['PRIMARY KEY', 'AUTO_INCREMENT']},
   {'name': 'full_name', 'type': 'VARCHAR(100)', 'constraints': ['NOT NULL']},
   {'name': 'email',
    'type': 'VARCHAR(100)',
    'constraints': ['NOT NULL', 'UNIQUE']},
   {'name': 'phone_numer',
    'type': 'VARCHAR(20)',
    'constraints': ['NOT NULL', 'UNIQUE']},
   {'name': 'dob', 'type': 'DATE', 'constraints': ['NOT NULL']},
   {'name': 'zip_code', 'type': 'VARCHAR(10)', 'constraints': ['NOT NULL']},
   {'name': 'createdAt',
    'type': 'DATETIME',
    'constraints': ['NOT N

In [3]:
knowledge_graph = pipeline.build_knowledge_graph(parsed_schema)

In [4]:
pipeline.visualize_kg_interactive(knowledge_graph, filename='link_kg1.html')

link_kg1.html


# Save Knowlege Graph

In [5]:
pipeline.save_kg_graphml(knowledge_graph, 'sql_knowledge_graph_link')

# Load Knowledge Graph

In [6]:
loaded_knowledge_graph = pipeline.load_kg_graphml('sql_knowledge_graph_link')

# Embed Query To KG

In [7]:
sql_query = """SELECT 
    U.full_name AS Full_name, 
    U.email AS Email, 
    UE.degree AS Degree, 
    E.university_name AS University, 
    YEAR(UE.graduation_date) AS Graduation_year, 
    UE.specialization AS Specialization
FROM User U
JOIN User_Education UE ON U.id = UE.user_id
JOIN Education E ON UE.education_id = E.id
WHERE UE.specialization LIKE 'DSAI%';
"""

In [8]:
parsed_query = pipeline.parse_sql(sql_query)
parsed_query

QueryComponents(raw_query="SELECT \n    U.full_name AS Full_name, \n    U.email AS Email, \n    UE.degree AS Degree, \n    E.university_name AS University, \n    YEAR(UE.graduation_date) AS Graduation_year, \n    UE.specialization AS Specialization\nFROM User U\nJOIN User_Education UE ON U.id = UE.user_id\nJOIN Education E ON UE.education_id = E.id\nWHERE UE.specialization LIKE 'DSAI%';\n", query_type='SELECT', tables={'User', 'Education', 'User_Education'}, columns={'Education.id', 'User_Education.degree', 'User.full_name', 'User.id', 'User_Education.user_id', 'User_Education.education_id', 'User.email', 'User_Education.graduation_date', 'User_Education.specialization', 'Education.university_name'}, joins=[{'source': 'User', 'target': 'User_Education', 'type': 'JOIN', 'conditions': [{'left': 'id', 'right': 'user_id', 'operator': '='}]}, {'source': 'User_Education', 'target': 'Education', 'type': 'JOIN', 'conditions': [{'left': 'education_id', 'right': 'id', 'operator': '='}]}], condit

In [9]:
knowledge_graph_update, query_id = pipeline.update_kg_with_query(loaded_knowledge_graph, parsed_query)

query_219575899597822764


In [10]:
query_id

'query_219575899597822764'

In [11]:
pipeline.visualize_query(knowledge_graph_update, query_id)

query_query_219575899597822764.html


# Extract Data From KG Query

In [12]:
pipeline.extract_query_labels(loaded_knowledge_graph, query_id)['tables']

[{'id': 'table_User', 'label': 'User'},
 {'id': 'table_Education', 'label': 'Education'},
 {'id': 'table_User_Education', 'label': 'User_Education'}]

In [13]:
pipeline.extract_query_labels(loaded_knowledge_graph, query_id)['columns']

[{'id': 'column_User.id', 'label': 'id', 'table': 'User'},
 {'id': 'column_Education.id', 'label': 'id', 'table': 'Education'},
 {'id': 'column_User_Education.user_id',
  'label': 'user_id',
  'table': 'User_Education'},
 {'id': 'column_Education.university_name',
  'label': 'university_name',
  'table': 'Education'},
 {'id': 'column_User.email', 'label': 'email', 'table': 'User'},
 {'id': 'column_User.full_name', 'label': 'full_name', 'table': 'User'},
 {'id': 'column_User_Education.education_id',
  'label': 'education_id',
  'table': 'User_Education'},
 {'id': 'column_User_Education.graduation_date',
  'label': 'graduation_date',
  'table': 'User_Education'},
 {'id': 'column_User_Education.degree',
  'label': 'degree',
  'table': 'User_Education'},
 {'id': 'column_User_Education.specialization',
  'label': 'specialization',
  'table': 'User_Education'}]